# Getting Started

## Creating DataFrames

In [5]:
# set filename(in hdfs)
userJSONFileName = "hdfs://hadoop:9000/data/ml-1m/users.json"
# read json file
df = spark.read.json(userJSONFileName)
# Displays the content of the DataFrame to stdout
df.show()

+---+------+----------+------+-------+
|age|gender|occupation|userID|zipcode|
+---+------+----------+------+-------+
|  1|     F|        10|     1|  48067|
| 56|     M|        16|     2|  70072|
| 25|     M|        15|     3|  55117|
| 45|     M|         7|     4|  02460|
| 25|     M|        20|     5|  55455|
| 50|     F|         9|     6|  55117|
| 35|     M|         1|     7|  06810|
| 25|     M|        12|     8|  11413|
| 25|     M|        17|     9|  61614|
| 35|     F|         1|    10|  95370|
| 25|     F|         1|    11|  04093|
| 25|     M|        12|    12|  32793|
| 45|     M|         1|    13|  93304|
| 35|     M|         0|    14|  60126|
| 25|     M|         7|    15|  22903|
| 35|     F|         0|    16|  20670|
| 50|     M|         1|    17|  95350|
| 18|     F|         3|    18|  95825|
|  1|     M|        10|    19|  48073|
| 25|     M|        14|    20|  55113|
+---+------+----------+------+-------+
only showing top 20 rows



## Untyped Dataset Operations (aka DataFrame Operations)

In [6]:
# Print the schema in a tree format
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- userID: long (nullable = true)
 |-- zipcode: string (nullable = true)

